In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tflearn
import re #regular expresions

from collections import Counter

# Load CSV File

In [113]:
csv = pd.read_csv("harmonized_system_utf_8.csv", dtype="str")

In [114]:
print(csv[3000:3005])


       Order Level short_code   long_code  long_parent    code parent  \
3000  928131     4     441510  4.4151E+11    4.415E+11  4415.1  44.15   
3001  928132     4     441520  4.4152E+11    4.415E+11  4415.2  44.15   
3002  928133     3     441600   4.416E+11  4.40021E+11    4416     44   
3003  928134     3     441700   4.417E+11  4.40021E+11    4417     44   
3004  928135     3     441800   4.418E+11  4.40021E+11   44.18     44   

                                            description  
3000  Cases. boxes. crates. drums and similar packin...  
3001  Pallets. box pallets and other load boards. of...  
3002  Casks. barrels. vats. tubs and other coopers' ...  
3003  Tools. tool bodies. tool handles. broom or bru...  
3004  Builders' joinery and carpentry. of wood. incl...  


# generate output hot encoder generator

In [115]:
def hot_encoder_generator(max_value):
    labels_hot_encoded = {}
    for i in range(max_value):
        zeros_array = np.zeros(max_value)
        zeros_array[i] = 1
        labels_hot_encoded[i] = zeros_array
    return labels_hot_encoded

hot_encoder = hot_encoder_generator(100)

## TODO: Flatten Hot encoded arrays

In [132]:
def flatten(array):
    return [val for sublist in array for val in sublist]

In [133]:
[hot_encoder[0], hot_encoder[10], hot_encoder[10]]

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

# Generate input word2vec 

In [134]:
count_level_1 = Counter()
count_level_2 = Counter()
count_level_3 = Counter()
word_counter = Counter()
for i, row in enumerate(csv.values):
    order, level, short_code, long_code, long_parent, code, parent, desc = row
    
    # if short_code is less than 5 add a 0 in the front
    if(len(short_code)==5): 
        short_code = "0" + short_code
    
    count_level_1[int(short_code[0:2])] += 1
    count_level_2[int(short_code[2:4])] += 1
    count_level_3[int(short_code[4:6])] += 1

    # 1. Create hot_encoded hs6
    
    # 2. Create word2vec description
    chars_to_remove = '[!@#$;,.-_()"\']'
    desc = re.sub(chars_to_remove, '', str(desc))
    desc = " ".join(desc.split())
    for word in desc.split(" "):
        word_counter[word] += 1
print(word_counter)
# print(word_counter.most_common())
# 3. Train NN
    
    
    
    


Counter({'or': 10771, 'of': 9741, 'and': 9185, 'excl': 3871, 'for': 2554, 'in': 2205, 'a': 1981, 'with': 1808, 'not': 1804, 'the': 1779, 'other': 1746, 'incl': 1668, 'yarn': 1605, 'containing': 1446, 'by': 1343, 'cotton': 1311, 'whether': 1201, 'fibres': 1103, 'spp': 1103, '%': 1081, 'weight': 1060, 'nan': 956, 'fabrics': 915, 'decitex': 854, 'similar': 810, 'to': 771, 'machines': 656, 'those': 636, 'cabled': 632, 'but': 579, 'nes': 564, 'articles': 548, 'leather': 532, 'on': 515, 'width': 508, 'heading': 505, 'paper': 503, 'rubber': 497, 'thread': 489, 'ncorhynchus': 476, 'steel': 474, 'products': 458, 'cm': 451, 'mm': 450, 'density': 442, 'thereof': 437, 'linear': 432, 'per': 419, 'crocheted': 419, 'metal': 417, 'made': 404, 'knitted': 401, 'up': 398, 'iron': 395, 'wood': 383, 'paperboard': 381, 'sheets': 380, 'materials': 380, 'sewing': 380, 'parts': 369, 'chilled': 368, 'pile': 361, 'used': 356, 'put': 344, 'without': 343, 'prepared': 343, 'textile': 332, 'predominantly': 329, 'sin

In [135]:
vocab = set(word_counter.keys())
vocab_size = len(vocab)
print(vocab_size)

word2index = {}

for i,word in enumerate(vocab):
    word2index[word] = i
word2index

11170


{'': 0,
 'filmm': 1,
 'felt': 2,
 'cichorium': 3,
 'rattans': 4,
 'papers': 5,
 'knittednted': 6,
 'callops': 7,
 'acetale': 8,
 'inches': 9,
 'sensitilm': 10,
 'spreads': 11,
 'screwing': 12,
 'ŒæŒæ': 13,
 'detonating': 14,
 'screens': 15,
 'visual': 16,
 'snowploughs': 17,
 'bazam': 18,
 'filletsresh': 19,
 'sparkling': 20,
 'plash': 21,
 'monoammonium': 22,
 'ulbs': 23,
 'adussh': 5613,
 'application': 26,
 'non-ionic': 25,
 'typres': 7055,
 'false': 29,
 'cracking': 28,
 'spongy': 30,
 'enthof': 31,
 'exclaphic': 32,
 'pipe-cutters': 36,
 'packing': 34,
 'cash': 35,
 'registers': 38,
 'fissile': 39,
 'thylene-propylene': 40,
 'aldehyde': 41,
 'flossreparations': 42,
 'polyethylene': 43,
 'being': 44,
 '-hexachlorocyclohexane': 45,
 'twine': 1885,
 'ferro-vanadium': 46,
 'whalebone': 47,
 'cupro-nickel': 48,
 'kale': 49,
 'bungs': 51,
 'prints': 1890,
 'colour': 52,
 'groves': 55,
 'insincl': 54,
 'weapons': 1888,
 'vinylbital': 56,
 'shell': 57,
 'hydraulic': 7476,
 'stran': 58,
 '

In [158]:
layer_0 = np.zeros((1,vocab_size))

def update_input_layer(desc):
    
    global layer_0
    
    # clear out previous state, reset the layer to be all 0s
    layer_0 *= 0
    for word in desc.split(" "):
        layer_0[0][word2index[word]] = 1

In [174]:
X = []
Y = []
for i, row in enumerate(csv.values):
    order, level, short_code, long_code, long_parent, code, parent, desc = row
    
    # if short_code is less than 5 add a 0 in the front
    if(len(short_code)==5): 
        short_code = "0" + short_code

    # 1. Create hot_encoded hs6
    output = flatten([hot_encoder[int(short_code[0:2])], hot_encoder[int(short_code[0:2])], hot_encoder[int(short_code[0:2])]])
    X.append(output)
    # 2. Create word2vec description
    chars_to_remove = '[!@#$;,.-_()"\']'
    desc = re.sub(chars_to_remove, '', str(desc))
    desc = " ".join(desc.split())
    for word in desc.split(" "):
        update_input_layer(desc)
    Y.append(flatten(layer_0))

# Create NN

In [175]:
def build_model():
    tf.reset_default_graph()
    
    #inputs
    net = tflearn.input_data([None, 300])
    
    #hidden layer
    net = tflearn.fully_connected(net, 15000, activation='ReLU')
    net = tflearn.fully_connected(net, 15000, activation='ReLU')
#     net = tflearn.fully_connected(net, 20, activation='ReLU')
    
    #output layer
    net = tflearn.fully_connected(net, 11170, activation='softmax')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.05, loss='categorical_crossentropy')
    model = tflearn.DNN(net)
    return model

In [ ]:
model = build_model()

In [ ]:
# training
model.fit(X, Y, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=40)


Training Step: 94  | total loss: 98.75201 | time: 1138.423s
| SGD | epoch: 002 | loss: 98.75201 - acc: 0.0042 -- iter: 5504/6495
